In [30]:
import os
from vanna.remote import VannaDefault
from dotenv import load_dotenv
from vanna.flask import VannaFlaskApp

In [31]:
load_dotenv()
api_key = os.getenv('VANNA_API_KEY')
vanna_model_name = os.getenv('VANNA_MODEL')
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

In [32]:
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT", "5432")  # default to 5432 if not set

vn.connect_to_postgres(
    host=db_host,
    dbname=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)


In [25]:
df_tables = vn.run_sql("""
SELECT
    table_catalog,
    table_schema,
    table_name
FROM INFORMATION_SCHEMA.TABLES
WHERE table_type = 'BASE TABLE'
  AND table_schema NOT IN ('pg_catalog', 'information_schema');
""")

for index, row in df_tables.iterrows():
    doc_line = f"Database: {row['table_catalog']} / Schema: {row['table_schema']} / Table: {row['table_name']}"
    vn.train(documentation=doc_line)

Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....


In [26]:
df_fk = vn.run_sql("""
SELECT
  tc.table_catalog,               -- add table_catalog here, too!
  tc.table_schema,
  tc.table_name,
  kcu.column_name,
  ccu.table_schema AS foreign_table_schema,
  ccu.table_name AS foreign_table_name,
  ccu.column_name AS foreign_column_name
FROM information_schema.table_constraints AS tc
JOIN information_schema.key_column_usage AS kcu
  ON tc.constraint_name = kcu.constraint_name
  AND tc.table_schema = kcu.table_schema
JOIN information_schema.constraint_column_usage AS ccu
  ON ccu.constraint_name = tc.constraint_name
  AND ccu.table_schema = tc.table_schema
WHERE tc.constraint_type = 'FOREIGN KEY'
  AND tc.table_schema NOT IN ('pg_catalog', 'information_schema');
""")

for index, row in df_fk.iterrows():
    doc_line = f"Database: {row['table_catalog']} / Schema: {row['table_schema']} / Table: {row['table_name']}"
    vn.train(documentation=doc_line)


Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....


In [18]:
vn.train(documentation="All the Captains, Agents, and Organization related people are in User table with different userTypes, and UserType is another Table which specify type of user. and Customers table consists of registered users, passengers table consists of passenegers who travelled with us and need not to be in customers.")
vn.train(documentation="The User table has all the information about the users, and the UserType table has the type of users. The Customers table has the information about the registered users, and the Passengers table has the information about the passengers who have traveled with us.")
vn.train(documentation="ReservedTickets table consists of all booked tickets and trip details, passenger details")
vn.train(documentation="TicketTransaction table consists of all the transactions related to tickets, and this table has all the payment details with necessary columns of what platform and when it got booked")
vn.train(documentation="ReservedTicketSeats table consists of and triplevel details, seatIds, passenger details of seat and so on")
vn.train(documentation="AgentPaymentTransaction table consists of all the transactions related to agents, and this table has all the payment details with necessary columns of what platform and when it got booked")
vn.train(documentation="Revenue should be calculated from ReservedTicketSeats table and not from TicketTransactions table")
vn.train(documentation="Blocked Seats will be in ReservedTicketSeats table with status as available = false and dummy tickets will also be in ReservedTicketSeats table with status as available = false and availableAt is not null and will be equal to currentTime of first TripboardingPoint of that trip")

Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....


'2988071-doc'

In [ ]:
vn.train(question="how many dummies are there for hyd-vjy 1500 service",sql='''
SELECT count(ts."seatId") as seatcount
FROM "Trips" t
join "Services" s on t."serviceId" = s.id
JOIN "TripSeats" ts ON t.id = ts."tripId"
JOIN "TripBoardingPoints" tbp on t.id = tbp."tripId" 
WHERE t."journeyDate" = current_date
  AND ts.available = FALSE
  AND ts."availableAt" = tbp."currentTime"
  and s."serviceNumber" = 'HYD-VJY-AC-SE-1500'
''')

In [ ]:
vn.train(question="what are the captain details for tpty bnglr 1400 service today",sql='''
SELECT uc."firstName" , uc."mobile"
FROM "User" uc
JOIN "Trips" t ON uc."id" = t."captainId"
WHERE t."sourceId" = 12 -- Assuming Bangalore (BLR) is represented by sourceId = 7
AND t."destinationId" = 7 -- Assuming Tirupati (TP) is represented by destinationId = 12
and t."serviceId" in (select id from "Services" where  "serviceNumber"='TP-BN-AC-SE-1400')
AND t."journeyDate" = CURRENT_DATE;
         ''')

In [ ]:
vn.train(question="How many Trips are running today from Hyd to vjy",sql='''SELECT COUNT(*) FROM "Trips"
WHERE "journeyDate" = current_date AND
"sourceId"=3 AND "destinationId"=5 AND "active" = true''')

In [ ]:
vn.train(question="How many Services are running today?",sql='''SELECT COUNT(*) FROM "Trips" WHERE "journeyDate" = current_date AND "active" = true''')

In [ ]:
vn.train(question="How many Tickets got booked today in hyd - vjy 1500 service?",sql=''' 
                                                SELECT COUNT(*) AS tickets_booked_today_hyderabad_to_vijayawada
                                                FROM "ReservedTicketSeats" rts
                                                join "ReservedTickets" rt on rt.id = rts."ticketId" 
                                                JOIN "Trips" t ON rt."tripId" = t.id
                                                join "Services" s on s.id = t."serviceId"
                                                WHERE t."journeyDate" = CURRENT_DATE
                                                AND s."serviceNumber"  = 'HYD-VJY-AC-SE-1500' AND rt."active" = true;
''')

In [ ]:
vn.train(question="How many passengers were boarded in VJY HYD 11:30-OR Service today?",
         sql='''
    select count("passengerId") from "ReservedTicketSeats" rts
join "ReservedTickets" rt on rt.id = rts."ticketId" 
where rt."serviceId" in (select id from "Services" s where s."serviceNumber" = 'VJY-HYD-AC-SE-1130OR')
and rts."isBoarded" = true and rt."tripId" in (select id from "Trips" t where t."journeyDate" = current_date) 
''')

In [ ]:
vn.train(question="How many seats got booked in VJY HYD 11:30-OR Service today?",
         sql='''
select count("passengerId") from "ReservedTicketSeats" rts
join "ReservedTickets" rt on rt.id = rts."ticketId" 
where rt."serviceId" in (select id from "Services" s where s."serviceNumber" = 'VJY-HYD-AC-SE-1130OR')
and rt."tripId" in (select id from "Trips" t where t."journeyDate" = current_date) 
''')

In [ ]:
vn.train(question="How many female seats were booked today in HYD VJY 1500 service?",
         sql='''
select count(*) from "TripSeats" ts 
join "Trips" t on t.id=ts."tripId" 
where ts.available = false 
and ts."availablityStatus" = 'F' 
and t."serviceId" in (select id from "Services" s where s."serviceNumber" = 'HYD-VJY-AC-SE-1500')
and t."journeyDate" = current_date
and "seatId" not in (select "seatId" from "TripSeats" where "available" = false 
         and "availableAt" = (SELECT "currentTime"
        FROM "TripBoardingPoints" tbp 
        JOIN "Trips" t ON t.id = tbp."tripId"
        JOIN "Services" s ON s.id = t."serviceId"
        WHERE t."journeyDate" = current_date
        AND t."serviceId" in (select id from "Services" s where s."serviceNumber" = 'HYD-VJY-AC-SE-1500')
        ORDER BY tbp."currentTime" ASC
        LIMIT 1))    
         
''')

In [ ]:
vn.train(question="How many tickets got booked by QR today?",
         sql='''
select * from "ReservedTicketSeats" rts 
join "ReservedTickets" rt on rt.id = rts."ticketId"  
where rt."subCategory" = 'QR' and rts.active = true 
and rt."tripId" = (select id from "Trips" t where t."journeyDate" = current_date and t."serviceId" = (select id from "Services" where "serviceNumber" = 'HYD-VJY-AC-SE-1500'))
''')

In [ ]:
vn.train(question="How many Green coins were used for this ticket 'FRE615998'?",
         sql='''
select gct.coins  from "TicketTransactions" tt 
join "ReservedTickets" rt on tt."ticketId" = rt.id
join "GreenCoinTransactions" gct on tt."greenCoinsTransactionId" = gct.id 
where rt."invoiceNumber" = 'FRE615998' 
''')

In [ ]:
vn.train(question="How many tickets were booked by agent whose firstName is 'RajaSekar Reddy' today?",
         sql='''
select count(*) from "TicketTransactions" tt
join "ReservedTickets" rt  on rt.id = tt."ticketId"
where rt."subCategory" = 'Agent' and rt.active = true 
and tt."createdById" = (select id from "User" u  where u."firstName" = 'Rajasekar Reddy')
and rt."tripId" in (select id from "Trips" t where t."journeyDate" = current_date)
and tt."transactionType" = 'Booked' 
''')

In [ ]:
vn.train(question="How many seats were booked by agent whose firstName is 'RajaSekar Reddy' today?", 
         sql= '''select count(rts.*) from "TicketTransactions" tt
join "ReservedTickets" rt  on rt.id = tt."ticketId"
join "ReservedTicketSeats" rts on rt.id = rts."ticketId"
where rt."subCategory" = 'Agent' and rt.active = true 
and tt."createdById" = (select id from "User" u  where u."firstName" = 'Rajasekar Reddy')
and rt."tripId" in (select id from "Trips" t where t."journeyDate" = current_date)
and tt."transactionType" = 'Booked' 
''')

In [ ]:
vn.train(question="How many tickets were cancelled yesterday?",
         sql= '''select COUNT(*) as count_cancelled_yesterday
from "TripCancellations" tc 
where tc."createdAt" = (CURRENT_TIMESTAMP - interval '1 day');''')

In [33]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn,title="FreshBus AI ChatBot",allow_llm_to_see_data=False,sql=False,csv_download=True)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
